In [37]:
from monai.utils import first, set_determinism
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Activations,
)

from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.data import CacheDataset, DataLoader, Dataset

import torch
import matplotlib.pyplot as plt

import os
from glob import glob
import numpy as np

from monai.inferers import sliding_window_inference

In [38]:
in_dir = "C:/Users/mihne/Desktop/Accenture-Collab/C3ScanServerFinal cu zip/C3ScanServerFinal/PersistanceLayer/1/1"
model_dir = "C:/Users/mihne/Desktop/Accenture-Collab/LiverSeg-using-monai-and-pytorch/Liver-Segmentation-Using-Monai-and-PyTorch/results"

In [39]:
path_train_volumes = sorted(glob(os.path.join(in_dir, "Data", "*.nii.gz")))
train_files = [{"vol": image_name,"seg":image_name} for image_name in path_train_volumes]


In [40]:
test_transforms = Compose(
    [
        LoadImaged(keys=["vol", "seg"]),
        AddChanneld(keys=["vol", "seg"]),
        Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
        Orientationd(keys=["vol", "seg"], axcodes="RAS"),
        ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200,b_min=0.0, b_max=1.0, clip=True), 
        CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
        Resized(keys=["vol", "seg"], spatial_size=[128,128,128]),   
        ToTensord(keys=["vol", "seg"]),
    ]
)

In [41]:
test_ds = Dataset(data=train_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1)

In [42]:
device = torch.device("cuda:0")
model = UNet(
    dimensions=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256), 
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

In [43]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()

UNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
        (unit1): Convolution(
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Sequential(


In [ ]:
sw_batch_size = 4
roi_size = (128, 128, 128)
with torch.no_grad():
    test_patient = first(test_loader)
    t_volume = test_patient['vol']
    #t_segmentation = test_patient['seg']
    
    test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
    sigmoid_activation = Activations(sigmoid=True)
    test_outputs = sigmoid_activation(test_outputs)
    test_outputs = test_outputs > 0.53
    l=[]
    for i in range(5):
        # plot the slice [:, :, 80]
        plt.figure("check", (18, 6))
        plt.subplot(1, 3, 1)
        plt.title(f"image {i}")
        plt.imshow(test_patient["vol"][0, 0, :, :, i], cmap="gray")
        plt.subplot(1, 3, 2)
        plt.title(f"output {i}")
        plt.imshow(test_outputs.detach().cpu()[0, 1, :, :, i])
        
        plt.savefig(f"lool/slice{i}.png")
        plt.clf()
        with open(f"lool/slice{i}.png", "rb") as image:
            f = image.read()
            b = bytearray(f)
            l.append(b)
    plt.close()

    
    

In [2]:
from monai.utils import first, set_determinism
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Activations,
)

from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.data import CacheDataset, DataLoader, Dataset

import torch
import matplotlib.pyplot as plt

import os
from glob import glob
import numpy as np

from monai.inferers import sliding_window_inference

In [5]:



def convert_to_png(in_dir,out_dir,model_dir,slice_number):
    patient_volume = sorted(glob(os.path.join(in_dir, "Data", "*.nii.gz")))
    patient_files = [{"vol": image_name, "seg": image_name} for image_name in patient_volume]
    patient_transform = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=(1.5, 1.5, 1.0), mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-200, a_max=200, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=[128, 128, slice_number]),
            ToTensord(keys=["vol", "seg"]),
        ]
    )

    patient_ds = Dataset(data=patient_files, transform=patient_transform)
    patient_loader = DataLoader(patient_ds, batch_size=1)

    device = torch.device("cuda:0")
    model = UNet(
        dimensions=3,
        in_channels=1,
        out_channels=2,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
        norm=Norm.BATCH,
    ).to(device)

    model.load_state_dict(torch.load(
        os.path.join(model_dir, "best_metric_model.pth")))

    sw_batch_size = 4
    roi_size = (128, 128, slice_number)
    with torch.no_grad():
        test_patient = first(patient_loader)
        t_volume = test_patient['vol']
        test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
        sigmoid_activation = Activations(sigmoid=True)
        test_outputs = sigmoid_activation(test_outputs)
        test_outputs = test_outputs > 0.53

        list_byte_arrays=[]
        for i in range(slice_number):
            # plot the slice [:, :, 80]
            plt.figure("check", (18, 6))
            plt.subplot(1, 2, 1)
            plt.title(f"image {i}")
            plt.imshow(test_patient["vol"][0, 0, :, :, i], cmap="gray")
            plt.subplot(1, 2, 2)
            plt.title(f"output {i}")
            plt.imshow(test_outputs.detach().cpu()[0, 1, :, :, i])
            plt.savefig(f"{out_dir}/mask{i}.png")
            plt.clf()
            with open(f"{out_dir}/mask{i}.png", "rb") as image:
                f = image.read()
                b = bytearray(f)
                list_byte_arrays.append(b)
        plt.close()
        return list_byte_arrays

images=convert_to_png('C:/Users/mihne/Desktop/Accenture-Collab/LiverSeg-using-monai-and-pytorch/Liver-Segmentation-Using-Monai-and-PyTorch/LiverSegementationDataSet/Converting and Stuff', 'Masks', 'C:/Users/mihne/Desktop/Accenture-Collab/LiverSeg-using-monai-and-pytorch/Liver-Segmentation-Using-Monai-and-PyTorch/results',32)


